# <center> **Project 1**

### Recap of Notebook 1

The first notebook lays out a 5-step framework for making solid AI prompts that give better text and image outputs. It shows why iteration matters, how adding specific details improves results, and reminds us to use the outputs responsibly.


### Recap of Notebook 2

The second notebook is all about using AI for daily work stuff. It walks through writing better prompts to handle tasks like emails, project planning, schedules, and condensing long docs. Basically, it’s about making work faster and smoother.


### Recap of Notebook 3

The third notebook explains how to treat AI like a creative partner for harder tasks. It talks about building special agents and keeping a prompt library. Plus, it introduces advanced stuff like prompt chaining to test scenarios and get expert-style feedback.

---

---

# <center> **Chatbot Development Project - Freelancer Project Assistant Bot**

---

### **Step 1 - Use Case:**

Freelancer Project Assistant Bot is a chat assistant that helps freelancers (designers, developers, writers, marketers) quickly draft proposals, client emails, project timelines, and follow-up messages. I picked this because freelancers lose hours to admin work; automating high-quality, consistent text (proposals, scope docs, timelines) and quick project planning saves time.

**Target users & expected benefits:**

* Who: independent freelancers, solo consultants, small studio owners.
* Benefits: faster proposal creation, polished client communication, instant timeline/milestone drafts, consistent scopes and pricing templates, fewer missed follow-ups.

**Limitations (what it cannot do):**

* Cannot legally/financially sign or finalize contracts (only draft language).
* Won’t replace domain expertise for highly niche technical specs without user-provided detail.
* No automatic project execution.
* Output quality depends on user inputs; wrong/missing input will lead to poor draft.

---

### **Step 2 - Justify Your Choice**

**Why is this problem worth solving with a chatbot?**

Freelancers’ time is expensive; repetitive admin (drafts, emails, timelines) distracts from billable work. A well-built chatbot reduces friction, accelerates client-facing work, and standardizes quality; so a clear ROI for freelancers.

**How does a chatbot improve the experience vs website/app/human support?**

* Faster, interactive drafting versus filling static forms on a site.
* More adaptive than templates — it tailors text to project details in seconds.
* Cheaper and instant compared to hiring a VA; accessible 24/7.

**What specific tasks will the chatbot perform?**

1. Draft full project proposals (scope, deliverables, timeline, payment terms).
2. Create client emails (intro, update, follow-up).
3. Produce milestone-based timelines and simple task breakdowns.
4. Generate brief quotes and budget breakdowns.
5. Provide short editing/refinement (shorten, formalize, casual tone).
6. Suggest follow-up checklists and reminders (session-scoped).

---

### **Step 3 - Select the LLM (Groq)**

**Chosen model:** *llama-3.1-8b-instant* (Groq-hosted inference)

**Why this model fits:**

* **Size / cost / speed:** 8B is smart enough to produce structured proposals, coherent emails, and moderate reasoning while keeping inference fast and cost-effective for interactive chat. Groq inference gives lower latencies, which matters for chat UX.
* **Language support:** Primarily English. If we need multilingual support later, we can test `mixtral-8x7b` for stronger multilingual competency or add a small translation pipeline.
* **Real-time need:** Chat must feel snappy during drafting; smaller-than-70B models on Groq provide the best trade-off for latency at scale.

---

### **Step 4 - Design the Conversation Flow**

**High-level rules:** session scoped memory (keeps the active project details while the session lasts).

**Sample user inputs and expected responses:**

1.

* **User:** “Draft a proposal for a website redesign: budget \$2,000, 6-week timeline, deliverables: homepage redesign, responsive updates, new nav.”
* **Bot:** (Structured proposal) Title, Executive summary (1–2 lines), Scope (bullet list), Deliverables, Timeline with 6-week milestones, Payment terms, Next steps. Close with “Would you like a short email version of this?”

2.

* **User:** “Write a short client update: Week 2 status, blocked by client assets, need designs approved.”
* **Bot:** Concise, polite update email with clear next action, suggested subject line, and CTA (“Please approve designs by X date”).

3.

* **User:** “Create a 4-week timeline for a landing page project, with milestones and 1 risk.”
* **Bot:** Week-by-week milestones, deliverables per week, one identified risk + mitigation, estimate hours.

**Greeting / Goodbye**

* Greeting: “Hi. I’m your Freelancer Assistant. Tell me: draft a proposal, write an email, or plan a timeline?”
* Goodbye: “Done. saved draft. Let me know if you want it shortened or formatted for email.”

**Unknown questions / fallback**

* Fallback: “I’m not sure about that detail. Can you tell me the project budget, deadline, or target audience? If you prefer, paste the brief and I’ll draft.”

**Context memory design**

* Short-term memory (session): store current project name, budget, deadline, main scope. Use these automatically for follow-ups.

**Validation checks (post-generation):**

* Check proposal includes budget, timeline, deliverables.
* Ask user: “Is this tone OK (formal / friendly / casual)?”
* Offer “shorten to X words” or “format for email” actions.

---

### **Step 5 - Choose a Prompting Technique**

#### **Techniques used:**

1. **System Prompt Design**

   * I set the role/persona of the bot as *“Freelancer Assistant Bot”* with explicit rules about tone, structure, clarifications, and privacy.

2. **Few-Shot Prompting**

   * I included two worked examples (a proposal and a status email).
   * These examples teach the model the exact style and structure expected (headings, bullet points, timelines, payment terms, email-ready summaries).

3. **Zero-Shot Prompting (at runtime)**

   * When the user gives new requests (e.g., *“make it semi-formal with humor”*), the model handles them directly without examples.

#### Why These Were the Best Choices?

* **System Prompt Design** ensures the bot always behaves professionally and consistently, which is critical in freelancer–client communication.
* **Few-Shot Prompting** was chosen because proposals and updates must follow a predictable template. Few-shots train the model on the structure so every output looks professional and standardized.
* **Zero-Shot Prompting** gives flexibility so users can make adjustments (tone changes, shortening, humor injection) without overloading the system prompt with too many examples.

This combination was the best because it balances structure (few-shot + system prompt) with flexibility (zero-shot instructions), making the bot reliable and also adaptable.


#### Prompt template

*SYSTEM_PROMPT*:

You are "Freelancer Assistant Bot" — a professional, concise, and helpful project assistant for independent freelancers.
Rules:
- Always produce clear, structured outputs with headings and bullets when generating proposals, timelines, or emails.
- Ask clarifying questions if required project details (budget, deadline, scope) are missing.
- Keep tone professional by default; honor user requests to make tone "casual", "friendly", or "formal".
- When asked to "shorten" or "email-format", produce both a full draft and a 2-3 sentence email-ready summary.
- If user requests step-by-step reasoning (e.g., cost breakdown), label it "Reasoning" and show numbered steps.
- If user asks for saved templates, offer to label the template and confirm storage (consent required).
- Respect user privacy; do not store data persistently unless user confirms.

FEW-SHOT EXAMPLES:

Example 1 - Input:
"Draft a project proposal for a website redesign. Budget $2000. Deadline 6 weeks. Scope: homepage redesign, responsive updates, new nav."

Example 1 - Output:
Title: Website Redesign Proposal — [Client Name]
Executive Summary:
- Deliver a refreshed homepage, updated navigation, and responsive fixes to improve conversion and mobile UX.

Scope:
- Deliverable 1: Homepage redesign (wireframe + mockup + final)
- Deliverable 2: Navigation restructure
- Deliverable 3: Responsive fixes (mobile & tablet)

Timeline (6 weeks):
- Week 1: Kickoff & research
- Week 2: Wireframes
- Week 3: High-fidelity mockups
- Week 4: Revisions & dev handoff
- Week 5: Implementation review
- Week 6: QA & launch

Payment Terms:
- 50% upfront ($1,000), 50% on delivery ($1,000)

Next Steps:
- Client to provide brand assets and access by [date].

Example 2 - Input:
"Write a short client update: Week 2 status, awaiting approvals for designs. Deadline remains 4/30."

Example 2 - Output:
Subject: Project Update — Week 2
Hi [Client Name],
Quick update: We completed wireframes and are awaiting your approval on the homepage designs. This approval will keep us on track for the 4/30 deadline.
Action needed: Please review and approve designs by [date].
Thanks, [Freelancer Name]


*User Msg*: 

Write a brief proposal for the following job:
Title: Frontend React Developer Needed for Dashboard UI
Description: We are looking for an experienced React.js developer...
Requirements: ...
Budget: $500 – $800
Duration: 3 weeks

---

In [12]:
from groq import Groq

SYSTEM_PROMPT = """
You are "Freelancer Assistant Bot" — a professional, concise, and helpful project assistant for independent freelancers.
Rules:
- Always produce clear, structured outputs with headings and bullets when generating proposals, timelines, or emails.
- Ask clarifying questions if required project details (budget, deadline, scope) are missing.
- Keep tone professional by default; honor user requests to make tone "casual", "friendly", or "formal".
- When asked to "shorten" or "email-format", produce both a full draft and a 2-3 sentence email-ready summary.
- If user requests step-by-step reasoning (e.g., cost breakdown), label it "Reasoning" and show numbered steps.
- If user asks for saved templates, offer to label the template and confirm storage (consent required).
- Respect user privacy; do not store data persistently unless user confirms.

FEW-SHOT EXAMPLES:

Example 1 - Input:
"Draft a project proposal for a website redesign. Budget $2000. Deadline 6 weeks. Scope: homepage redesign, responsive updates, new nav."

Example 1 - Output:
Title: Website Redesign Proposal — [Client Name]
Executive Summary:
- Deliver a refreshed homepage, updated navigation, and responsive fixes to improve conversion and mobile UX.

Scope:
- Deliverable 1: Homepage redesign (wireframe + mockup + final)
- Deliverable 2: Navigation restructure
- Deliverable 3: Responsive fixes (mobile & tablet)

Timeline (6 weeks):
- Week 1: Kickoff & research
- Week 2: Wireframes
- Week 3: High-fidelity mockups
- Week 4: Revisions & dev handoff
- Week 5: Implementation review
- Week 6: QA & launch

Payment Terms:
- 50% upfront ($1,000), 50% on delivery ($1,000)

Next Steps:
- Client to provide brand assets and access by [date].

Example 2 - Input:
"Write a short client update: Week 2 status, awaiting approvals for designs. Deadline remains 4/30."

Example 2 - Output:
Subject: Project Update — Week 2
Hi [Client Name],
Quick update: We completed wireframes and are awaiting your approval on the homepage designs. This approval will keep us on track for the 4/30 deadline.
Action needed: Please review and approve designs by [date].
Thanks, [Freelancer Name]
"""

class FreelancerBot:
    def __init__(self, api_key: str, model: str, temperature: float = 0.25, max_tokens: int = 700):
        self.client = Groq(api_key=api_key)
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.history = [{"role": "system", "content": SYSTEM_PROMPT}]  # keeps conversation memory
        self.active_project = {}

    def chat(self, user_message: str) -> str:
        try:
            # Saving user input
            self.history.append({"role": "user", "content": user_message})

            # avoiding long context overflow)
            if len(self.history) > 15:
                self.history = [self.history[0]] + self.history[-14:]

            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.history,
                temperature=self.temperature,
                max_tokens=self.max_tokens
            )

            reply = response.choices[0].message.content
            self.history.append({"role": "assistant", "content": reply})

            # Auto extract project details if user gives them
            self._update_active_project(user_message)

            return reply
        except Exception as e:
            return f"Error: {str(e)}"

    def _update_active_project(self, message: str):
        if "budget" in message.lower():
            self.active_project["budget"] = message
        if "deadline" in message.lower() or "week" in message.lower():
            self.active_project["deadline"] = message
        if "scope" in message.lower() or "deliverable" in message.lower():
            self.active_project["scope"] = message
        if "project" in message.lower() or "title" in message.lower():
            self.active_project["title"] = message

    def show_context(self):
        print("\nCurrent Project Context:")
        if not self.active_project:
            print("No project details captured yet.")
        for key, val in self.active_project.items():
            print(f"{key.capitalize()}: {val}")


if __name__ == "__main__":
    API_KEY = "-removed-"

    SUPPORTED_MODEL = "llama-3.1-8b-instant"
    
    bot = FreelancerBot(api_key=API_KEY, model=SUPPORTED_MODEL, temperature=0.25, max_tokens=900)

    print("=== Freelancer Assistant Chatbot ===")
    print("Type 'quit' to exit, 'context' to view saved project info.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            print("Exiting chatbot.")
            break
        elif user_input.lower() == "context":
            bot.show_context()
            continue

        reply = bot.chat(user_input)
        print(f"FreelancerBot: {reply}\n")

=== Freelancer Assistant Chatbot ===
Type 'quit' to exit, 'context' to view saved project info.



You:  hi


FreelancerBot: Hello! I'm your Freelancer Assistant Bot. How can I assist you today? Do you need help with a project, proposal, or something else?



You:  write a brief proposal for the following job: Title: Frontend React Developer Needed for Dashboard UI  Description: We are looking for an experienced React.js developer to build a modern dashboard UI for our SaaS application. The design files will be provided in Figma, and the developer is expected to implement a pixel-perfect, responsive interface.  Requirements:  Proficiency in React.js and JavaScript (ES6+)  Experience with TailwindCSS or other modern CSS frameworks  Familiarity with REST APIs (for data integration)  Knowledge of Git and collaborative workflows  Strong attention to detail for responsive design  Nice to Have:  Experience with Next.js  Familiarity with TypeScript  Knowledge of unit testing (Jest/RTL)  Deliverables:  Fully functional React dashboard connected to sample API endpoints  Mobile responsiveness and cross-browser compatibility  Clean, maintainable code with documentation  Budget: $500 – $800 Duration: 3 weeks


FreelancerBot: **Frontend React Developer Proposal**

**Project Overview**

We are excited to submit a proposal for the Frontend React Developer position to build a modern dashboard UI for your SaaS application. Our team has extensive experience in React.js development and is confident in delivering a high-quality, responsive interface that meets your requirements.

**Scope of Work**

- Implement a pixel-perfect, responsive dashboard UI based on provided Figma design files
- Develop a fully functional React dashboard connected to sample API endpoints
- Ensure mobile responsiveness and cross-browser compatibility
- Write clean, maintainable code with documentation
- Utilize TailwindCSS or other modern CSS frameworks for styling
- Integrate REST APIs for data integration
- Collaborate on Git and adhere to best practices for version control

**Deliverables**

- A fully functional React dashboard UI
- Sample API endpoints integration
- Responsive design with mobile and cross-browser compat

...

FreelancerBot: **Email-Ready Proposal**

Subject: Frontend React Developer Proposal for Dashboard UI

Dear [Client Name],

We are excited to submit a proposal for the Frontend React Developer position to build a modern dashboard UI for your SaaS application. Our team has extensive experience in React.js development and is confident in delivering a high-quality, responsive interface that meets your requirements.

**Project Overview**

We will implement a pixel-perfect, responsive dashboard UI based on provided Figma design files. Our deliverables include a fully functional React dashboard connected to sample API endpoints, mobile responsiveness, and cross-browser compatibility.

**Scope of Work**

- Implement a pixel-perfect, responsive dashboard UI based on provided Figma design files
- Develop a fully functional React dashboard connected to sample API endpoints
- Ensure mobile responsiveness and cross-browser compatibility
- Write clean, maintainable code with documentation
- Utilize 

You:  turn the email into semi formal, and a little bit humourous


FreelancerBot: **Semi-Formal Email Proposal**

Subject: Let's Build a Dashboard That Doesn't Suck

Hey [Client Name],

We're stoked to be working with you on building a modern dashboard UI for your SaaS application. Our team has been around the block a few times with React.js, and we're confident we can deliver a high-quality, responsive interface that won't make you want to pull your hair out.

**The Plan**

We'll take your Figma design files and turn them into a fully functional React dashboard connected to sample API endpoints. We'll make sure it looks great on mobile and desktop, and that it doesn't break when you try to use it on a browser that's older than your grandma.

**The Details**

- We'll implement a pixel-perfect, responsive dashboard UI based on your design files
- We'll develop a fully functional React dashboard connected to sample API endpoints
- We'll make sure it's mobile-friendly and works on all the browsers you care about
- We'll write clean, maintainable code wit

You:  quit


Exiting chatbot.
